## Final Project Submission

Please fill out:
* Student name: Sean Fry  
* Student pace: self paced
* Scheduled project review date/time: 1/24
* Instructor name: Claude Fried
* Blog post URL:


# Movie Genre, Gross Earnings, and Rating Analysis

## Overview

This project is analyzing the data from imdb and Box Office Mojo, which has data on over 3000 movies. We are looking at what specific genres of movies along with ratings are trending to have the most box office earnings. Microsoft will be able to use my data analysis to determine which direction to take a new movie studio to have a higher chance of early successes.

## Business Problem
Recommend genre, runtime, and release month

## Data Understanding
imdb and Box Office Mojo have data on thousands of movies, separated by titles or an ID specific to each movie. The data files provide information on the years the movies were made, the genres, runtime, rating, and gross earnings.

In [33]:
import pandas as pd
import numpy as np

In [111]:
basics = pd.read_csv('./zippedData/imdb.title.basics.csv.gz')
ratings = pd.read_csv('./zippedData/imdb.title.ratings.csv.gz')
gross = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
movies = pd.read_csv('./zippedData/tmdb.movies.csv.gz')

In [112]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [113]:
gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [114]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [115]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


### Gross Data
This data contains the domestic and foreign gross earnings for movies separated by titles.

In [116]:
gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


### Movie Basic Info
This data contains the title, year made, runtime, and genres of the movies. The movies are listed by an internal ID number

In [117]:
basics.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


### Ratings Data
This data contains the number of ratings and the average rating. The movies are separated by an internal ID number.

In [118]:
ratings.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


### Data Preparation
I'm taking each data set and normalizing column names and dropping extra columns. For the gross table, I need to make a total gross column and make sure it is a numeric value.

In [119]:
gross['foreign_gross'].fillna(value=0, inplace=True)

In [120]:
#Check to make sure I got rid of null values
gross['foreign_gross'].isnull().sum()

0

In [121]:
#Getting rid of commas in the gross columns
gross['foreign_gross'] = gross['foreign_gross'].apply(lambda x: float(str(x).replace(',','')))

In [122]:
#Making total Gross column to compare worldwide.
gross['tot_gross'] = gross['domestic_gross'] + gross['foreign_gross']
gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   3387 non-null   float64
 4   year            3387 non-null   int64  
 5   tot_gross       3359 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 158.9+ KB


In [123]:
gross.head()

,title,studio,domestic_gross,foreign_gross,year,tot_gross
0,Toy Story 3,BV,415000000.0,652000000.0,2010,1.067000e+09
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,1.025500e+09
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010,9.603000e+08
3,Inception,WB,292600000.0,535700000.0,2010,8.283000e+08
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,7.526000e+08


Cleaning up the genres of the basics column so that each movie will show in each genre it is in during analysis.

In [124]:
basics['genres'] = basics['genres'].apply(lambda x: str(x).replace(',', ' '))

In [125]:
basics['genres'] = basics['genres'].apply(lambda x: x.split())

In [126]:
basics.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"[Action, Crime, Drama]"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"[Biography, Drama]"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,[Drama]
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"[Comedy, Drama]"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"[Comedy, Drama, Fantasy]"


In [127]:
movies.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [128]:
#Changing date to date-time format
movies['release_date'] = pd.to_datetime(movies['release_date'])

In [129]:
#Extracting the month from the movies dataframe into a new column
movies['release_month'] = movies['release_date'].apply(lambda x: str(x.month))

In [130]:
## Changes the primary_title column to just title so we can join with the gross table more easily.
basics = basics.rename(columns={'primary_title': 'title'})

In [131]:
## Dropping columns that are not relevant to what I am looking for.
basics.drop(columns = ['original_title', 'start_year'], inplace=True)
gross.drop(columns = ['domestic_gross', 'foreign_gross', 'studio', 'year'], inplace=True)
movies.drop(columns = ['Unnamed: 0', 
                       'genre_ids', 'id', 
                       'original_language', 'original_title',
                       'popularity', 'vote_average', 'vote_count'], 
            inplace=True)

### Merging Data Sets
I'll be merging all of the data into one table, keeping titles, genres, rating, and gross income. When I'm done I'll filter out any movies with less than 100 reviews.

In [132]:
#merging the imdb tables together first
basics_and_ratings = pd.merge(basics, ratings, on=['tconst'], how='inner')
basics_and_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73856 entries, 0 to 73855
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           73856 non-null  object 
 1   title            73856 non-null  object 
 2   runtime_minutes  66236 non-null  float64
 3   genres           73856 non-null  object 
 4   averagerating    73856 non-null  float64
 5   numvotes         73856 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 3.9+ MB


In [133]:
# Keeping the duplicate rows witht the highest review numbers.
basics_and_ratings.sort_values(by=['title', 'numvotes'], inplace=True)

In [134]:
basics_and_ratings = basics_and_ratings.drop_duplicates(subset=['title'])

In [135]:
basics_and_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69993 entries, 7706 to 45325
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           69993 non-null  object 
 1   title            69993 non-null  object 
 2   runtime_minutes  62563 non-null  float64
 3   genres           69993 non-null  object 
 4   averagerating    69993 non-null  float64
 5   numvotes         69993 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 3.7+ MB


In [136]:
#merging the combined gross and release table
date_and_gross = pd.merge(gross, movies, on=['title'], how='inner')

In [137]:
date_and_gross.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2703 entries, 0 to 2702
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          2703 non-null   object        
 1   tot_gross      2682 non-null   float64       
 2   release_date   2703 non-null   datetime64[ns]
 3   release_month  2703 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 105.6+ KB


In [140]:
data = pd.merge(date_and_gross, basics_and_ratings, on=['title'], how='inner')

In [141]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2606 entries, 0 to 2605
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   title            2606 non-null   object        
 1   tot_gross        2589 non-null   float64       
 2   release_date     2606 non-null   datetime64[ns]
 3   release_month    2606 non-null   object        
 4   tconst           2606 non-null   object        
 5   runtime_minutes  2558 non-null   float64       
 6   genres           2606 non-null   object        
 7   averagerating    2606 non-null   float64       
 8   numvotes         2606 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 203.6+ KB


In [142]:
data.head()

,title,tot_gross,release_date,release_month,tconst,runtime_minutes,genres,averagerating,numvotes
0,Toy Story 3,1.067000e+09,2010-06-17,6,tt0435761,103.0,"[Adventure, Animation, Comedy]",8.3,682218
1,Inception,8.283000e+08,2010-07-16,7,tt1375666,148.0,"[Action, Adventure, Sci-Fi]",8.8,1841066
2,Shrek Forever After,7.526000e+08,2010-05-16,5,tt0892791,93.0,"[Adventure, Animation, Comedy]",6.3,167532
3,The Twilight Saga: Eclipse,6.985000e+08,2010-06-23,6,tt1325004,124.0,"[Adventure, Drama, Fantasy]",5.0,211733
4,Iron Man 2,6.239000e+08,2010-05-07,5,tt1228705,124.0,"[Action, Adventure, Sci-Fi]",7.0,657690


In [150]:
data.drop(columns=['release_date', 'tconst', 'numvotes'], inplace=True)

In [144]:
#dropping duplicate titles, keeping earlier release month
data.sort_values(by=['title', 'release_month'], inplace=True)

In [145]:
data = data.drop_duplicates(subset=['title'])

In [151]:
data.sort_index()

,title,tot_gross,release_month,runtime_minutes,genres,averagerating
0,Toy Story 3,1.067000e+09,6,103.0,"[Adventure, Animation, Comedy]",8.3
1,Inception,8.283000e+08,7,148.0,"[Action, Adventure, Sci-Fi]",8.8
2,Shrek Forever After,7.526000e+08,5,93.0,"[Adventure, Animation, Comedy]",6.3
3,The Twilight Saga: Eclipse,6.985000e+08,6,124.0,"[Adventure, Drama, Fantasy]",5.0
4,Iron Man 2,6.239000e+08,5,124.0,"[Action, Adventure, Sci-Fi]",7.0
...,...,...,...,...,...,...
2600,Loving Pablo,2.200000e+04,6,123.0,"[Action, Biography, Crime]",6.3
2602,The Escape,1.400000e+04,10,107.0,"[Drama, Thriller]",7.3
2603,Souvenir,1.140000e+04,9,86.0,"[Comedy, Romance]",5.9
2604,The Quake,6.200000e+03,12,106.0,"[Action, Drama, Thriller]",6.2


In [152]:
data.to_csv('./zippedData/clean_data.csv')

## Analysis
We are going to look at three things. 
1.)Average earnings by release month. This will be a bar graph grouping movies by release month
    and averaging their earnings.
2.)Average earnings by genre. Same as above, but with genre as the x groupings.
3.)Average earnings by runtime. Line graph(?) average earnings by runtime. Make a function to find $/min

In [23]:
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [167]:
data['release_month'] = data['release_month'].apply(lambda x: int(x))

In [169]:
#aggregating money data by release month
month_money = data[['release_month', 'tot_gross']].groupby('release_month').agg(['count', 'mean', 'median']).sort_values(by='release_month')

In [174]:
month_money

tot_gross                          
                  count          mean      median
release_month                                    
1                   154  6.452344e+07  23150000.0
2                   136  8.463733e+07  28267349.5
3                   176  7.213392e+07  10622500.0
4                   184  5.206184e+07   8350000.0
5                   174  1.319170e+08   5550000.0
6                   171  1.532182e+08  12400000.0
7                   166  1.498916e+08  30787000.0
8                   170  6.449726e+07  17200000.0
9                   203  4.998691e+07   5479000.0
10                  240  4.470449e+07   3200000.0
11                  213  1.192822e+08  21735000.0
12                  207  1.247154e+08  26100000.0